<a href="https://colab.research.google.com/github/Nanangk/Crack_Classification_with_Transfer_Learning_VGG16_in_Keras/blob/master/Classifier_with_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Crack Classification with Transfer Learning VGG16</h1>

#Import Libraries and Packages

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

Finally, we will be leveraging the VGG16 model to build our classifier, so let's download it as well.

In [6]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

#Download Data

In [7]:
## get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip

--2021-06-26 16:02:10--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261482368 (249M) [application/zip]
Saving to: ‘concrete_data_week3.zip.1’

concrete_data_week3 100%[===================>] 249.37M  29.3MB/s    in 8.8s    

2021-06-26 16:02:19 (28.4 MB/s) - ‘concrete_data_week3.zip.1’ saved [261482368/261482368]



In [ ]:
!unzip concrete_data_week3.zip

## Define Global Constants

In [13]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

## Construct ImageDataGenerator Instances

In [14]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [15]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [16]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 10001 images belonging to 2 classes.


## Build, Compile and Fit Model

In [17]:
model = Sequential()

In [18]:
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

58900480/58889256 [==============================] - 1s 0us/step


In [19]:
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

In [20]:
model.layers

In [21]:
model.layers[0].layers

In [22]:
model.layers[0].trainable = False

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 14,784,546
Trainable params: 69,858
Non-trainable params: 14,714,688
_______________________________________

In [24]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [26]:
steps_per_epoch_training

301

In [27]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
301/301 [==============================] - 239s 602ms/step - loss: 0.1344 - accuracy: 0.9453 - val_loss: 0.0062 - val_accuracy: 0.9979
Epoch 2/2
301/301 [==============================] - 181s 600ms/step - loss: 0.0101 - accuracy: 0.9972 - val_loss: 0.0045 - val_accuracy: 0.9987


In [ ]:
model.save('classifier_vgg16_model.h5')